In [ ]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy
from functools import reduce

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageModeler import OutageModeler
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer
from CustomJSON import CustomEncoder, CustomWriter

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras

import scipy

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# ---------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------
# Parameters: General
#----------------------------------------------------------------------------------------------------
# save_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models'
save_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240907\Models'
#-----
# save_sub_dir  = None
# save_sub_dir = 'All_EEMSP_agg_Top10_v5'
save_sub_dir = 'All_EEMSP_agg_Top10_noMonth_20250219'
#-----
include_prbl = True
verbose      = True
force_fresh_data_build = False

In [ ]:
#----------------------------------------------------------------------------------------------------
# Parameters: Building/grabbing MECPOCollection objects (and merged_dfs, etc.)
#----------------------------------------------------------------------------------------------------
# The following can be set individually for _outg/_otbl/_prbl if needed:
#     acq_run_date, data_date_ranges, data_dir_base
#   e.g., one can use acq_run_date to set for all, or acq_run_date_outg to set _outg individually
# Slightly special case:
#  grp_by_cols_bsln can be used to set grp_by_cols_otbl and grp_by_cols_prbl, but grp_by_cols_outg must be set separately
#---------------------------------------------------------------------------
save_data = True

acq_run_date              = '20240907'
data_date_ranges          = [
    ['2023-04-01', '2024-08-31'], 
]
#-----
cpx_dfs_name              = 'rcpo_df_norm_by_xfmr_nSNs'
#-----
data_dir_base             = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
#-----
grp_by_cols_outg          = ['outg_rec_nb', 'trsf_pole_nb']
grp_by_cols_bsln          = ['trsf_pole_nb', 'no_outg_rec_nb']
#-----
std_dict_grp_by_cols_outg = {
    'outg_rec_nb'  : 'outg_rec_nb', 
    'trsf_pole_nb' : 'trsf_pole_nb'
}
std_dict_grp_by_cols_bsln = {
    'no_outg_rec_nb'  : 'outg_rec_nb', 
    'trsf_pole_nb'    : 'trsf_pole_nb'
}
#---------------------------------------------------------------------------

In [ ]:
# # THESE SHOULD BE ADDED TO MODEL_SUMMARY_DICT AND AUTOMATICALLY EXTRACTED!!!!!
# freq                        = '5D', 
# group_cols                  = ['trsf_pole_nb'], 
# date_col                    = 'aep_event_dt', 
# normalize_by_SNs            = True, 
# include_power_down_minus_up = False, 
# regex_patterns_to_remove    = ['.*cleared.*', '.*Test Mode.*'], 
# combine_cpo_df_reasons      = True, 
# include_n_eemsp             = True

In [ ]:
#---------------------------------------------------------------------------
freq='5D'
days_min_max_outg_td_windows=[
    [1,6], [6,11], [11,16], [16,21], [21,26], [26,31]
]
old_to_new_keys_dict = None
#-------------------------
normalize_by_time_interval  = True
#-------------------------
include_power_down_minus_up = False
#-------------------------
regex_to_remove_patterns    = ['.*cleared.*', '.*Test Mode.*']
regex_to_remove_ignore_case = True
#-------------------------
max_total_counts = None
# max_total_counts=150
# max_total_counts={
#     '01-05 Days':150, 
#     '06-10 Days':150, 
#     '11-15 Days':150, 
#     '16-20 Days':150,
#     '21-25 Days':150, 
#     '26-30 Days':150
# }
how_max_total_counts='any'
#-------------------------  
mecpo_idx_for_ordering = 0
#------------------------- 

In [ ]:
#----------------------------------------------------------------------------------------------------
# Parameters: Building the final train/test/holdout datasets and running model fit
#----------------------------------------------------------------------------------------------------
save_model = True

# random_state = None
random_state = 42
#-------------------------
n_top_reasons_to_inclue = 10
# n_top_reasons_to_inclue = None
combine_others          = True
#-------------------------
merge_eemsp = True
eemsp_mult_strategy='agg'
#-------------------------
include_month = False
#-------------------------
an_keys_to_drop = None
# an_keys_to_drop = ['00-01 Days']
# an_keys_to_drop = ['01-06 Days']
#-------------------------
date_0_train   = pd.to_datetime('2023-04-01')
date_1_train   = pd.to_datetime('2023-11-30')
#-----
date_0_test    = pd.to_datetime('2023-04-01')
date_1_test    = pd.to_datetime('2023-11-30')
#-----
date_0_holdout = None
date_1_holdout = None

#-------------------------
test_size                = 0.33
get_train_test_by_date   = False
split_train_test_by_outg = True 
#-------------------------
create_validation_set = False
val_size              = 0.10 #w.r.t to train size (i.e., w.r.t 1.0-test_size)
#-------------------------
run_scaler=True
#-------------------------
run_PCA = False
pca_n_components=0.95
#-------------------------
remove_others_from_outages=False
#-------------------------
# min_pct_target_1 = 25
min_pct_target_1 = None
#-------------------------
reduce_train_size = False
red_test_size = 0.75 #Amount kept will be 1.0-red_test_size
#-------------------------
outgs_slicer = OutageModeler.get_dummy_slicer()

In [ ]:
outg_mdlr_0 = OutageModeler(
    include_prbl                 = include_prbl, 
    save_base_dir                = save_base_dir, 
    save_sub_dir                 = save_sub_dir, 
    force_fresh_data_build       = force_fresh_data_build, 
    verbose                      = verbose, 
    #
    acq_run_date                 = acq_run_date, 
    data_date_ranges             = data_date_ranges, 
    cpx_dfs_name                 = cpx_dfs_name, 
    data_dir_base                = data_dir_base, 
    grp_by_cols_outg             = grp_by_cols_outg,   
    grp_by_cols_bsln             = grp_by_cols_bsln, 
    std_dict_grp_by_cols_outg    = std_dict_grp_by_cols_outg, 
    std_dict_grp_by_cols_bsln    = std_dict_grp_by_cols_bsln, 
    #
    freq                         = freq, 
    days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
    old_to_new_keys_dict         = None, 
    normalize_by_time_interval   = normalize_by_time_interval, 
    include_power_down_minus_up  = include_power_down_minus_up, 
    regex_to_remove_patterns     = regex_to_remove_patterns, 
    regex_to_remove_ignore_case  = regex_to_remove_ignore_case, 
    max_total_counts             = max_total_counts, 
    how_max_total_counts         = how_max_total_counts, 
    mecpo_idx_for_ordering       = mecpo_idx_for_ordering, 
    #
    random_state                 = random_state, 
    n_top_reasons_to_inclue      = n_top_reasons_to_inclue, 
    combine_others               = combine_others, 
    merge_eemsp                  = merge_eemsp, 
    eemsp_mult_strategy          = eemsp_mult_strategy, 
    include_month                = include_month, 
    an_keys_to_drop              = an_keys_to_drop, 
    date_0_train                 = date_0_train, 
    date_1_train                 = date_1_train, 
    date_0_test                  = date_0_test, 
    date_1_test                  = date_1_test, 
    date_0_holdout               = date_0_holdout, 
    date_1_holdout               = date_1_holdout, 
    test_size                    = test_size, 
    get_train_test_by_date       = get_train_test_by_date, 
    split_train_test_by_outg     = split_train_test_by_outg, 
    create_validation_set        = create_validation_set, 
    val_size                     = val_size, 
    run_scaler                   = run_scaler, 
    run_PCA                      = run_PCA, 
    pca_n_components             = pca_n_components,
    remove_others_from_outages   = remove_others_from_outages, 
    min_pct_target_1             = min_pct_target_1, 
    reduce_train_size            = reduce_train_size, 
    red_test_size                = red_test_size, 
    outgs_slicer                 = outgs_slicer, 
)
outg_mdlr_0.compile_data(
    assert_can_model  = True, 
    print_dumb_scores = True, 
    verbose           = True
)

#-----
# Initialize model and run fit
#-----

outg_mdlr_0.init_model_clf(
    model_type = 'random_forest', 
    n_estimators = 1000, 
    max_depth    = 25, 
    criterion    = 'gini',  
    class_weight = None,   
    n_jobs       = None, 
)

# outg_mdlr_0.init_model_clf(
#     model_type = 'random_forest', 
#     n_estimators = 10, 
#     max_depth    = 5, 
#     criterion    = 'gini',  
#     class_weight = None,   
#     n_jobs       = None, 
# )

outg_mdlr_0.fit(verbose=True)
outg_mdlr_0.predict(verbose=True)

In [ ]:
assert(0)

In [ ]:
# outg_mdlr_0.save_model()
# outg_mdlr_0.save_summary_dict()

In [ ]:
#--------------------------------------------------
trsf_loc_info_df = outg_mdlr_0.compile_trsf_location_info_df()
#-------------------------
trsf_loc_info_df = DOVSOutages.remove_trsf_pole_nb_without_numeric_digit(
    df               = trsf_loc_info_df, 
    trsf_pole_nb_col = 'trsf_pole_nb'
)
#-------------------------
# The dataset doesn't really matter, nor does the zip+4, so drop
trsf_loc_info_df = trsf_loc_info_df.drop(columns=['dataset', 'zip+4']).drop_duplicates().reset_index(drop=True)
#-------------------------
# For our purposes here, we want each trsf_pole_nb to correspond to exactly one zip code.
# The location of the trsf_pole_nb is inferred from the connected meters, meaning that some poles
#   can span multiple zip codes.
# To solve this, for each trsf_pole_nb, we will simply keep only the entry with the largest nPNs value.
# Things are complicated slightly by the fact that we also track the date_pd now, for cases where multiple
#   date periods are combined for an analysis.
# Therefore, for each (trsf_pole_nb, date_pd) we will keep the entry with the largest nPNs value.
# This should remain stable over different date periods, but it will be interesting to check this in the future
trsf_loc_info_df = trsf_loc_info_df.loc[trsf_loc_info_df.groupby(['trsf_pole_nb', 'date_pd'])['nPNs_xfmri_sa4i'].idxmax()].reset_index(drop=True)
assert(trsf_loc_info_df[['trsf_pole_nb', 'date_pd']].value_counts().shape[0] == trsf_loc_info_df.shape[0])

In [ ]:
trsf_loc_info_df['county_nm'].unique()

In [ ]:
trsf_loc_info_df['county_nm'].value_counts()

In [ ]:
trsf_loc_info_df

In [ ]:
zip_i = '43214'
trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['zip']==zip_i]['trsf_pole_nb'].unique().tolist()

zip_slcr_i = DFSlicer(
    single_slicers      = [dict(
        column              = ('index', 'trsf_pole_nb'), 
        value               = trsf_pole_nbs_i, 
        comparison_operator = 'isin'
    )], 
    name                = zip_i, 
    apply_not           = False, 
    join_single_slicers = 'and'    
)

outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i)
#-----
# In general, need a way of determining which functions need called
outg_mdlr_i.finalize_data(
    assert_can_model  = True, 
    print_dumb_scores = True, 
    verbose           = True
)
outg_mdlr_i.init_model_clf()
outg_mdlr_i.fit()
outg_mdlr_i.predict()

In [ ]:
outg_mdlr_i.df_test

In [ ]:
assert(0)

In [ ]:
zip_pop_df = pd.read_pickle(r'C:\Users\s346557\Downloads\zip_pop_df.pkl')
#-----
zip_clusters_df = zip_pop_df.groupby(['cluster_5']).agg({
    'zip' : list, 
    'population' : 'sum'
})
zip_clusters_df = zip_clusters_df.rename(columns={'zip':'zips'})
#-----
zip_clusters_df

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
trsf_loc_info_df = trsf_loc_info_df.loc[~trsf_loc_info_df['trsf_pole_nb'].isin(['NETWORK', 'PRIMARY', ' '])]

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
trsf_loc_info_df

In [ ]:
trsf_loc_info_df_wgrps = pd.merge(
    trsf_loc_info_df, 
    zip_clusters_df.explode('zips').reset_index().drop(columns=['population']), 
    how      = 'left', 
    left_on  = 'zip', 
    right_on = 'zips'
)

In [ ]:
trsf_loc_info_df_wgrps['cluster_5'].value_counts()

In [ ]:
# # OLD
# ledger = dict()
# for i_clstr, row_i in zip_clusters_df.iterrows():
#     zips_i = row_i['zips']
#     print(i_clstr)
#     print(zips_i)
#     print()
#     try:
#         trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['zip'].isin(zips_i)]['trsf_pole_nb'].unique().tolist()
#         #-----
#         # Each trsf_pole_nb should be mapped to only one model
#         assert(set(ledger.keys()).intersection(set(trsf_pole_nbs_i))==set())
#         #-----
#         # Update the ledger with the new pole_nb/cluster information
#         ledger.update({pole_j:i_clstr for pole_j in trsf_pole_nbs_i})        
#         zip_slcr_i = DFSlicer(
#             single_slicers      = [dict(
#                 column              = ('index', 'trsf_pole_nb'), 
#                 value               = trsf_pole_nbs_i, 
#                 comparison_operator = 'isin'
#             )], 
#             name                = str(zips_i), 
#             apply_not           = False, 
#             join_single_slicers = 'and'    
#         )
        
#         outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i, save_model=True, new_save_sub_dir=str(i_clstr))
#         #-----
#         # In general, need a way of determining which functions need called
#         outg_mdlr_i.build_train_test_data()
#         outg_mdlr_i.finalize_train_test_data()
#         outg_mdlr_i.init_model_clf()
#         outg_mdlr_i.fit()
#         outg_mdlr_i.predict()
#     except:
#         print('FAILED!!!!!')
#     print('\n'*5)

In [ ]:
ledger = zip_pop_df.set_index('zip')['cluster_5'].to_dict()
for i_clstr, row_i in zip_clusters_df.iterrows():
    zips_i = row_i['zips']
    print(i_clstr)
    print(zips_i)
    print()
    try:
        trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['zip'].isin(zips_i)]['trsf_pole_nb'].unique().tolist()
        #-----    
        zip_slcr_i = DFSlicer(
            single_slicers      = [dict(
                column              = ('index', 'trsf_pole_nb'), 
                value               = trsf_pole_nbs_i, 
                comparison_operator = 'isin'
            )], 
            name                = str(zips_i), 
            apply_not           = False, 
            join_single_slicers = 'and'    
        )
        
        outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i, save_model=True, new_save_sub_dir=str(i_clstr))
        #-----
        # In general, need a way of determining which functions need called
        outg_mdlr_i.finalize_data(
            assert_can_model  = False, 
            print_dumb_scores = True, 
            verbose           = True
        )
        if not outg_mdlr_i.can_model:
            print('Cannot model!!!!! Skipping')
            print('\n'*5)
            continue
        outg_mdlr_i.init_model_clf()
        outg_mdlr_i.fit()
        outg_mdlr_i.predict()
    except:
        print('FAILED!!!!!')
    print('\n'*5)

In [ ]:
outg_mdlr_i.save_base_dir

In [ ]:
CustomWriter.output_dict_to_json(
    os.path.join(outg_mdlr_i.save_base_dir, 'ledger.json'), 
    ledger
)

In [ ]:
# outg_mdlr_0.save_model()
# outg_mdlr_0.save_summary_dict()

In [ ]:
outg_mdlr_i.save_base_dir

In [ ]:
zip_i = '43235'
trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['zip']==zip_i]['trsf_pole_nb'].unique().tolist()

zip_slcr_i = DFSlicer(
    single_slicers      = [dict(
        column              = ('index', 'trsf_pole_nb'), 
        value               = trsf_pole_nbs_i, 
        comparison_operator = 'isin'
    )], 
    name                = zip_i, 
    apply_not           = False, 
    join_single_slicers = 'and'    
)

outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i)
#-----
# In general, need a way of determining which functions need called
outg_mdlr_i.build_train_test_data()
outg_mdlr_i.finalize_train_test_data()
outg_mdlr_i.init_model_clf()
outg_mdlr_i.fit()
outg_mdlr_i.predict()

In [ ]:
zip_i = '43235'
trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['zip']==zip_i]['trsf_pole_nb'].unique().tolist()

zip_slcr_i = DFSlicer(
    single_slicers      = [dict(
        column              = ('index', 'trsf_pole_nb'), 
        value               = trsf_pole_nbs_i, 
        comparison_operator = 'isin'
    )], 
    name                = zip_i, 
    apply_not           = False, 
    join_single_slicers = 'and'    
)

outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i)
#-----
# In general, need a way of determining which functions need called
outg_mdlr_i.build_train_test_data()
outg_mdlr_i.finalize_train_test_data()
outg_mdlr_i.init_model_clf(
    model_type = 'random_forest', 
    n_estimators = 100, 
    max_depth    = 25, 
    criterion    = 'gini',  #'gini' or 'entropy', 
    class_weight = None,   # None or 'balanced'
    n_jobs       = None,     
)
outg_mdlr_i.fit()
outg_mdlr_i.predict()

In [ ]:
zip_i = '43148'
trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['zip']==zip_i]['trsf_pole_nb'].unique().tolist()

zip_slcr_i = DFSlicer(
    single_slicers      = [dict(
        column              = ('index', 'trsf_pole_nb'), 
        value               = trsf_pole_nbs_i, 
        comparison_operator = 'isin'
    )], 
    name                = zip_i, 
    apply_not           = False, 
    join_single_slicers = 'and'    
)

outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i)
#-----
# In general, need a way of determining which functions need called
outg_mdlr_i.build_train_test_data()
outg_mdlr_i.finalize_train_test_data()
outg_mdlr_i.init_model_clf()
outg_mdlr_i.fit()
outg_mdlr_i.predict()

In [ ]:
outg_mdlr_i.merged_df_outg

In [ ]:
outg_mdlr_i.merged_df_otbl

In [ ]:
outg_mdlr_i.time_infos_df

In [ ]:
city = 'COLUMBUS'
trsf_pole_nbs_i = trsf_loc_info_df[trsf_loc_info_df['city']==city]['trsf_pole_nb'].unique().tolist()

zip_slcr_i = DFSlicer(
    single_slicers      = [dict(
        column              = ('index', 'trsf_pole_nb'), 
        value               = trsf_pole_nbs_i, 
        comparison_operator = 'isin'
    )], 
    name                = city, 
    apply_not           = False, 
    join_single_slicers = 'and'    
)

outg_mdlr_i = outg_mdlr_0.slice_off_model(zip_slcr_i)
#-----
# In general, need a way of determining which functions need called
outg_mdlr_i.build_train_test_data()
outg_mdlr_i.finalize_train_test_data()
outg_mdlr_i.init_model_clf(
    model_type = 'random_forest', 
    n_estimators = 100, 
    max_depth    = 10, 
    criterion    = 'gini',  #'gini' or 'entropy', 
    class_weight = None,   # None or 'balanced'
    n_jobs       = None,     
)
outg_mdlr_i.fit()
outg_mdlr_i.predict()

In [ ]:
trsf_loc_info_df[trsf_loc_info_df['city']==city]['zip'].unique()

In [ ]:
idk = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v5\data_structure_df.pkl')

In [ ]:
natsorted(idk.columns.get_level_values(1).unique())

In [ ]:
idk

### 